In [1]:
import nomic
import json
nomic.login("nk-2LSEn5FB5h04Dn0pFLuZ9jDw7ZaMbPpYPV88wbKKASc")
#- Scientific programming
import numpy as np
import pandas as pd
from tqdm import tqdm
import os


from nomic import AtlasDataset

from unidecode import unidecode
import re
import datetime

dataset = AtlasDataset('hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08')

df = dataset.maps[0].data.df

congress_name_list = list(np.unique(list(df['name'])))
congress_handles = [list(df['twitter_lower'][df['name'] == cn])[0] for cn in congress_name_list]


df_congress = pd.DataFrame({'twitter_lower': congress_handles, 'congress_name_list': congress_name_list})
df_congress = df_congress.merge(df.drop_duplicates(subset='twitter_lower'), on='twitter_lower', how='left') ## ignore the column <115><2017><senate><179> and <118><2024><house><142>

2024-10-24 09:31:36.950 | INFO     | nomic.dataset:__init__:768 - Loading existing dataset `hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08`.
2024-10-24 09:31:37.740 | WARNING  | nomic.data_operations:df:923 - Converting to pandas dataframe. This may materialize a large amount of data into memory.
2024-10-24 09:31:37.742 | INFO     | nomic.data_operations:_download_data:902 - Downloading data
100%|██████████| 129/129 [00:01<00:00, 86.95it/s] 
2024-10-24 09:31:46.181 | INFO     | nomic.data_operations:_load_data:872 - Loading data
100%|██████████| 129/129 [00:27<00:00,  4.73it/s]


In [2]:
df_congress.head()

,twitter_lower,congress_name_list,tweetId,postedAt,name,state,party,chamber,years,source,id,text
0,repfinkenauer,Abby Finkenauer D-IA,1266367737145683968,2020-05-29 13:56:11.167,Abby Finkenauer D-IA,IA,D,House,2019_2020,PolitWoops,2010694,Minor league baseball is a vital community ins...
1,repspanberger,Abigail Spanberger D-VA,1377973148675665920,2021-04-02 13:16:14.804,Abigail Spanberger D-VA,VA,D,House,2021_2022,PolitWoops,2511843,The #AmericanRescuePlan will provide funds to ...
2,repkinzinger,Adam Kinzinger R-IL,837318315202719744,2017-03-02 15:06:57.623,Adam Kinzinger R-IL,IL,R,House,2017_2018,PolitWoops,1141249,"Thanks, @Joepa84 - I currently serve in the 11..."
3,repadamschiff,Adam Schiff D-CA,493881370440392704,2014-07-28 22:11:09.242,Adam Schiff D-CA,CA,D,House,2013_2014,PolitWoops,376089,Concerned about #IdentityTheft? I'm hosting a ...
4,repadamsmith,Adam Smith D-WA,1511117188941459456,2022-04-04 23:03:07.700,Adam Smith D-WA,WA,D,House,2021_2022,PolitWoops,2753835,"Tonight the House is voting on H.R. 1916, the ..."


In [3]:
import os

file_list = os.listdir('/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Bill Votes/1024')


for k in range(len(file_list)):
    file_name = file_list[k]
    read_file_path = '/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Bill Votes/1024/' + file_name
    bill_name = file_name.split("congress_votes_")[1].split(".csv")[0]
    
    df_vote_result=pd.read_csv(read_file_path, header=1)
    rep_or_senate=df_vote_result['name'][0].split(' [')[0].split('. ')[0]
    
    df_vote_result['matched-name'] = ''
    df_vote_result['twitter-handle'] = ''
    df_vote_result['matched-state'] = ''


    for i, row in df_vote_result.iterrows():
        name = unidecode(row['name'].split(' [')[0].split(rep_or_senate+'. ')[1])
        party_val = row['party'][0]
        state_val = row['state']
        candidate_names = df_congress.loc[(df_congress['party'] == party_val) & (df_congress['state'] == state_val), 'congress_name_list'].apply(unidecode)
        name_parts = name.split(" ")

        for napa in name_parts:
            if candidate_names.str.contains(napa).any():
                index = candidate_names[candidate_names.str.contains(napa)].index[0]


                df_vote_result.at[i, 'matched-name'] = df_congress.loc[index]['congress_name_list']
                df_vote_result.at[i, 'twitter-handle'] = df_congress.loc[index]['twitter_lower']
                df_vote_result.at[i, 'matched-party']=df_congress.loc[index]['party']
                df_vote_result.at[i, 'matched-state']=df_congress.loc[index]['state']
            else:
                df_vote_result.at[i, 'matched-name'] = np.nan
                df_vote_result.at[i, 'twitter-handle'] = np.nan
                df_vote_result.at[i, 'matched-party']=np.nan
                df_vote_result.at[i, 'matched-state']=np.nan
    
    
    
    save_file_path = '/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Matched Bill Votes/1024'

    df_vote_result.to_csv(os.path.join(save_file_path, bill_name + '.csv'), index=False)


    print(bill_name)

/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


118-2023_h131


/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


118-2023_h243


/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


...

/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


118-2024_h145


/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


118-2024_h242
118-2024_h437


/tmp/ipykernel_1165034/807501312.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_1165034/807501312.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


In [19]:
df_vote_result=pd.read_csv(read_file_path, header=1)
df_vote_result.head()

,person,state,district,vote,name,party
0,400004,AL,4,Nay,"Rep. Robert Aderholt [R, 1997-2024]",Republican
1,400030,GA,2,Yea,"Rep. Sanford Bishop [D, 1993-2024]",Democrat
2,400033,OR,3,Yea,"Rep. Earl Blumenauer [D, 1996-2024]",Democrat
3,400046,TX,8,Nay,"Rep. Kevin Brady [R, 1997-2022]",Republican
4,400052,TX,26,Nay,"Rep. Michael Burgess [R, 2003-2024]",Republican


In [20]:
rep_or_senate=df_vote_result['name'][0].split(' [')[0].split('. ')[0]
print(rep_or_senate)

Rep


In [21]:
df_vote_result['matched-name'] = ''
df_vote_result['twitter-handle'] = ''
df_vote_result['matched-state'] = ''


for i, row in df_vote_result.iterrows():
    name = unidecode(row['name'].split(' [')[0].split(rep_or_senate+'. ')[1])
    party_val = row['party'][0]
    state_val = row['state']
    candidate_names = df_congress.loc[(df_congress['party'] == party_val) & (df_congress['state'] == state_val), 'congress_name_list'].apply(unidecode)
    name_parts = name.split(" ")

    for napa in name_parts:
        if candidate_names.str.contains(napa).any():
            index = candidate_names[candidate_names.str.contains(napa)].index[0]


            df_vote_result.at[i, 'matched-name'] = df_congress.loc[index]['congress_name_list']
            df_vote_result.at[i, 'twitter-handle'] = df_congress.loc[index]['twitter_lower']
            df_vote_result.at[i, 'matched-party']=df_congress.loc[index]['party']
            df_vote_result.at[i, 'matched-state']=df_congress.loc[index]['state']
        else:
            df_vote_result.at[i, 'matched-name'] = np.nan
            df_vote_result.at[i, 'twitter-handle'] = np.nan
            df_vote_result.at[i, 'matched-party']=np.nan
            df_vote_result.at[i, 'matched-state']=np.nan

df_vote_result.head()



,person,state,district,vote,name,party,matched-name,twitter-handle,matched-state,matched-party
0,400004,AL,4,Nay,"Rep. Robert Aderholt [R, 1997-2024]",Republican,"Aderholt, Robert",robert_aderholt,AL,R
1,400030,GA,2,Yea,"Rep. Sanford Bishop [D, 1993-2024]",Democrat,"Bishop Jr., Sanford",sanfordbishop,GA,D
2,400033,OR,3,Yea,"Rep. Earl Blumenauer [D, 1996-2024]",Democrat,"Blumenauer, Earl",repblumenauer,OR,D
3,400046,TX,8,Nay,"Rep. Kevin Brady [R, 1997-2022]",Republican,Kevin Brady R-TX,repkevinbrady,TX,R
4,400052,TX,26,Nay,"Rep. Michael Burgess [R, 2003-2024]",Republican,"Burgess, Michael",michaelcburgess,TX,R


## Warning: there will be unmatched CPs:

In [22]:
na_rows = df_vote_result[df_vote_result['matched-name'].isna()]
print(na_rows)


     person state  district vote  \
9    400075    SC         6  Yea   
30   400230    RI         2  Yea   
73   400616    NC         1  Yea   
77   400640    NE         1  Nay   
78   400641    NY        26  Yea   
105  412239    VT         0  Yea   
146  412443    MS         4  Nay   
153  412470    RI         1  Yea   
158  412486    WA         3  Nay   
179  412536    IL        13  Nay   
221  412623    GA        10  Nay   
396  456820    IL        15  Nay   
411  456835    NM         3  Yea   

                                              name       party matched-name  \
9          Rep. James “Jim” Clyburn [D, 1993-2024]    Democrat          NaN   
30        Rep. James “Jim” Langevin [D, 2001-2022]    Democrat          NaN   
73   Rep. George “G.K.” Butterfield [D, 2004-2022]    Democrat          NaN   
77            Rep. Jeff Fortenberry [R, 2005-2022]  Republican          NaN   
78               Rep. Brian Higgins [D, 2013-2024]    Democrat          NaN   
105                Re

In [23]:

save_file_path = '/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Matched Bill Votes/Hayden Choice'

df_vote_result.to_csv(os.path.join(save_file_path, bill_name + '.csv'), index=False)
